# AI Math Olympiad with [KerasNLP](https://github.com/keras-team/keras-nlp) and [Keras](https://github.com/keras-team/keras)

<div align="center">
    <img src="https://i.ibb.co/9rx4pbX/AIMO.png">
</div>

In this competition, we aim is to build AI models that can solve tough math problems, in other words, creating LLM models capable of solving Math Olympiad problems. This notebook will guide you through the process of fine-tuning the **Gemma** LLM model with LoRA to solve math problems using KerasNLP. With KerasNLP, fine-tuning with LoRA becomes straightforward with just a few lines of code.

**Did you know:**: This notebook is backend-agnostic? Which means it supports TensorFlow, PyTorch, and JAX backends. However, the best performance can be achieved with `JAX`. KerasNLP and Keras enable the choice of preferred backend. Explore further details on [Keras](https://keras.io/keras_3/).

**Note**: For a deeper understanding of KerasNLP, refer to the [KerasNLP guides](https://keras.io/keras_nlp/).


# Install Libraries

We need to install latest KerasNLP to load Gemma 1.1 model. As we don't have access to internet during inference, we will be installing this library from our local files.


In [2]:
!pip install -q /kaggle/input/keras-lib-dataset/keras_nlp-0.9.2-py3-none-any.whl --no-deps

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/kaggle/input/keras-lib-dataset/keras_nlp-0.9.2-py3-none-any.whl'



# Import Libraries


In [4]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9" # avoid memory fragmentation on JAX backend.

import keras
import keras_nlp

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown

# Configuration


In [5]:
class CFG:
    seed = 42
    dataset_path = "/kaggle/input/ai-mathematical-olympiad-prize"
    preset = "gemma_1.1_instruct_2b_en" # name of pretrained Gemma
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    epochs = 1 # number of epochs to train

# Reproducibility

Sets value for random seed to produce similar result in each run.


In [6]:
keras.utils.set_random_seed(CFG.seed)

# Data

No training data is provided in this competition; in other words, we can use any openly available datasets for this competition. In this notebook, we will use a modified **Math** dataset which I have compiled to have a `Question-Solution-Answer` format.

**Data Format:**

These datasets include:

- `problem`: The math problem in LaTeX format.
- `solution`: Step-by-step solution to this problem.
- `answer`: Final answer of the solution which will be the ground truth for this competition.
- `level`: Difficulty of the problem.
- `type`: The category of the problem.

> This dataset comes with its own train test split. However, we will merge them both and use them for fine-tuning. You are welcome to use them for trainining and validation separately. Also to reduce the training time we will only be training on the first`1000` samples. You are welcome to train on the full data.


In [9]:
df1 = pd.read_csv("/kaggle/input/math-qsa-dataset/train.csv")
df2 = pd.read_csv("/kaggle/input/math-qsa-dataset/test.csv")
# df = pd.concat([df1, df2], axis=0)
# df = df[:1000] # take first 1000 samples
# df.head(2)

df = df1

# Filter Data

The Math dataset contains various problems, but not all of them are suitable for this competition. More specifically, this competition requires a `non-negative integer` answer, while the Math dataset includes problems with different types of answers such as integers, floats, fractions, matrices, etc. In this notebook, we will only use those problems whose answers are non-negative integers and filter out the rest.


In [13]:
def is_integer(text):
    try:
        if int(text) >= 0:
            return True
        else:
            return False
    except ValueError:
        return False
    
df["is_integer"] = df.answer.map(is_integer)
df = df[df.is_integer].reset_index(drop=True)
df.head(2)

,problem,level,type,solution,answer,is_integer
0,The United States Postal Service charges an ex...,Level 3,Prealgebra,We calculate the desired ratio for each envelo...,3,True
1,How many integers between 1000 and 2000 have a...,Level 4,Prealgebra,"A number with 15, 20 and 25 as factors must be...",3,True


# Prompt Engineering

We will be using below simple prompt template we'll use to create problem-solution-answer trio to feed the model. This template will help the model to follow instruction and respond accurately. You can explore more advanced prompt templates for better results.

```
Role:
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.

Instruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.

Problem:
...

Solution:
...

Answer:
...
```


In [14]:
template = """Role:\nYou are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.\n\nInstruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.\n\nProblem:\n{problem}\n\nSolution:\n{solution}"""

In [15]:
df["prompt"] = df.progress_apply(lambda row: template.format(problem=row.problem,
                                                             solution=f"{row.solution}\n\nAnswer:\n{row.answer}"),
                                                             axis=1)
data = df.prompt.tolist()

  0%|          | 0/4456 [00:00<?, ?it/s]

In [19]:
len(data)

4456

Let's examine a sample prompt. As the answers in our dataset are curated with **markdown** format, we will render the sample using `Markdown()` to properly visualize the formatting.


## Check Sample


In [20]:
def colorize_text(text):
    for word, color in zip(["Role", "Instruction", "Problem", "Solution", "Answer"],
                           ["blue", "yellow", "red", "cyan", "green"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

## Sample 1


In [21]:
# Take a random sample
sample = data[12]

# Give colors to Instruction, Response and Category
sample = colorize_text(sample)

# Show sample in markdown
display(Markdown(sample))



**<font color='blue'>Role:</font>**
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.



**<font color='yellow'>Instruction:</font>**
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.



**<font color='red'>Problem:</font>**
What is the largest positive multiple of $12$ that is less than $350?$



**<font color='cyan'>Solution:</font>**
Dividing $350$ by $12$ gives a quotient $29$ with a remainder of $2$. In other words, \[350=12\cdot29+2.\]Thus, $29\cdot12=\boxed{348}$ is the largest multiple of $12$ which is less than $350.$



**<font color='green'>Answer:</font>**
348

## Sample 2


In [22]:
# Take a random sample
sample = data[32]

# Give colors to Instruction, Response and Category
sample = colorize_text(sample)

# Show sample in markdown
display(Markdown(sample))



**<font color='blue'>Role:</font>**
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.



**<font color='yellow'>Instruction:</font>**
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.



**<font color='red'>Problem:</font>**
We are given that $$54+(98\div14)+(23\cdot 17)-200-(312\div 6)=200.$$Now, let's remove the parentheses:  $$54+98\div14+23\cdot 17-200-312\div 6.$$What does this expression equal?



**<font color='cyan'>Solution:</font>**
Notice how the parentheses are only around pairs of numbers that are being multiplied or divided. Since multiplication and division are performed before addition and subtraction, it doesn't matter if we remove the parentheses. That's why  \begin{align*}
&54+(98\div14)+(23\cdot 17)-200-(312\div 6)\\
&=54+98\div14+23\cdot17-200-312\div 6\\
&=\boxed{200}.\end{align*}



**<font color='green'>Answer:</font>**
200

# Modeling

<div align="center"><img src="https://i.ibb.co/Bqg9w3g/Gemma-Logo-no-background.png" width="300"></div>

**Gemma** is a collection of advanced open LLMs developed by **Google DeepMind** and other **Google teams**, derived from the same research and technology behind the **Gemini** models. They can be integrated into applications and run on various platforms including mobile devices and hosted services. Developers can customize Gemma models using tuning techniques to enhance their performance for specific tasks, offering more targeted and efficient generative AI solutions beyond text generation.

Gemma models are available in several sizes so we can build generative AI solutions based on your available computing resources, the capabilities you need, and where you want to run them.

| Parameters size | Tuned versions    | Intended platforms                   | Preset                     |
| --------------- | ----------------- | ------------------------------------ | -------------------------- |
| 2B              | Pretrained        | Mobile devices and laptops           | `gemma_2b_en`              |
| 2B              | Instruction tuned | Mobile devices and laptops           | `gemma_1.1_instruct_2b_en` |
| 2B              | Pretrained        | Code Completion in Mobile Device     | `code_gemma_2b_en`         |
| 7B              | Pretrained        | Desktop computers and small servers  | `gemma_7b_en`              |
| 7B              | Instruction tuned | Desktop computers and small servers  | `gemma_1.1_instruct_7b_en` |
| 7B              | Instruction tuned | Code Completion in Desktop computers | `code_gemma_7b_en`         |

In this notebook, we will utilize the `Gemma 1.1 2b-it` model from KerasNLP's pretrained models to solve the math olympiad questions. We are using the "Instruction tuned" model instead of the "Pretrained" one because it is easier for the model to fine-tune on the prepared dataset.

To explore other available models, you can simply adjust the `preset` value in the `CFG` (config). You can find a list of other pretrained models on the [KerasNLP website](https://keras.io/api/keras_nlp/models/).


## Gemma Causal LM

The code below will build an end-to-end Gemma model for causal language modeling (hence the name `GemmaCausalLM`). A causal language model (LM) predicts the next token based on previous tokens. This task setup can be used to train the model unsupervised on plain text input or to autoregressively generate plain text similar to the data used for training. This task can be used for pre-training or fine-tuning a Gemma model simply by calling `fit()`.

This model has a `generate()` method, which generates text based on a prompt. The generation strategy used is controlled by an additional sampler argument on `compile()`. You can recompile the model with different `keras_nlp.samplers` objects to control the generation. By default, `"greedy"` sampling will be used.

> The `from_preset` method instantiates the model from a preset architecture and weights.


In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(CFG.preset)
gemma_lm.summary()

## Gemma LM Preprocessor

An important part of the Gemma model is the **Preprocessor** layer, which under the hood uses **Tokenizer**.

**What it does:** The preprocessor takes input strings and transforms them into a dictionary (`token_ids`, `padding_mask`) containing preprocessed tensors. This process starts with tokenization, where input strings are converted into sequences of token IDs.

**Why it's important:** Initially, raw text data is complex and challenging for modeling due to its high dimensionality. By converting text into a compact set of tokens, such as transforming `"The quick brown fox"` into `["the", "qu", "##ick", "br", "##own", "fox"]`, we simplify the data. Many models rely on special tokens and additional tensors to understand input. These tokens help divide input and identify padding, among other tasks. Making all sequences the same length through padding boosts computational efficiency, making subsequent steps smoother.

Explore the following pages to access the available preprocessing and tokenizer layers in **KerasNLP**:

- [Preprocessing](https://keras.io/api/keras_nlp/preprocessing_layers/)
- [Tokenizers](https://keras.io/api/keras_nlp/tokenizers/)


In [ ]:
x, y, sample_weight = gemma_lm.preprocessor(data[0:2])

This preprocessing layer will take in batches of strings, and return outputs in a `(x, y, sample_weight)` format, where the `y` label is the next token id in the `x` sequence.

From the code below, we can see that, after the preprocessor, the data shape is `(num_samples, sequence_length)`.


In [ ]:
# Display the shape of each processed output
for k, v in x.items():
    print(k, ":", v.shape)

# Inference before Fine-Tuning

Before we do fine-tuning, let's see how Gemma model responds with some prepared prompts.

> As this model is not yet fine-tuned for instruction, you will notice that the model's responses are inaccurate.


## Sample 1


In [ ]:
# Take one sample
row = df.iloc[12]

# Generate Prompt using template
prompt = template.format(
    problem=row.problem,
    solution="",
)

# Infer
output = gemma_lm.generate(prompt, max_length=1024)

# Colorize
output = colorize_text(output)

# Display in markdown
display(Markdown(output))


## Sample 2


In [ ]:
# Take one sample
row = df.iloc[32]

# Generate Prompt using template
prompt = template.format(
    problem=row.problem,
    solution=""
)

# Infer
output = gemma_lm.generate(prompt, max_length=1024)

# Colorize
output = colorize_text(output)

# Display in markdown
display(Markdown(output))


# Fine-tuning with LoRA

To get better responses from the model, we will fine-tune the model with Low Rank Adaptation (LoRA).

**What exactly is LoRA?**

LoRA is a method used to fine-tune large language models (LLMs) in an efficient way. It involves freezing the weights of the LLM and injecting trainable rank-decomposition matrices.

Imagine in an LLM, we have a pre-trained dense layer, represented by a $d \times d$ weight matrix, denoted as $W_0$. We then initialize two additional dense layers, labeled as $A$ and $B$, with shapes $d \times r$ and $r \times d$, respectively. Here, $r$ denotes the rank, which is typically **much smaller than** $d$. Prior to LoRA, the model's output was computed using the equation $output = W_0 \cdot x + b_0$, where $x$ represents the input and $b_0$ denotes the bias term associated with the original dense layer, which remains frozen. After applying LoRA, the equation becomes $output = (W_0 \cdot x + b_0) + (B \cdot A \cdot x)$, where $A$ and $B$ denote the trainable rank-decomposition matrices that have been introduced.

<center><img src="https://i.ibb.co/DWsbhLg/LoRA.png" width="300"><br/>
Credit: <a href="https://arxiv.org/abs/2106.09685">LoRA: Low-Rank Adaptation of Large Language Models</a> Paper</center>

In the LoRA paper, $A$ is initialized with $\mathcal{N} (0, \sigma^2)$ and $B$ with $0$, where $\mathcal{N}$ denotes the normal distribution, and $\sigma^2$ is the variance.

**Why does LoRA save memory?**

Even though we're adding more layers to the model with LoRA, it actually helps save memory. This is because the smaller layers (A and B) have fewer parameters to learn compared to the big model and fewer trainable parameters mean fewer optimizer variables to store. So, even though the overall model might seem bigger, it's actually more efficient in terms of memory usage.

> This notebook uses a LoRA rank of `4`. A higher rank means more detailed changes are possible, but also means more trainable parameters.


In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

**Notice** that, the number of trainable parameters is reduced from ~$2.5$ billions to ~$1.3$ millions after enabling LoRA.


## Training


In [ ]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = CFG.sequence_length 

# Compile the model with loss, optimizer, and metric
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=2e-5),
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train model
gemma_lm.fit(data, epochs=CFG.epochs, batch_size=CFG.batch_size)

# Inference after fine-tuning

Let's see how our fine-tuned model responds to the same questions we asked before fine-tuning the model.


## Sample 1


In [ ]:
# Take one sample
row = df.iloc[12]

# Generate Prompt using template
prompt = template.format(
    problem=row.problem,
    solution=""
)

# Infer
output = gemma_lm.generate(prompt, max_length=1024)

# Colorize
output = colorize_text(output)

# Display in markdown
display(Markdown(output))

## Sample 2


In [ ]:
# Take one sample
row = df.iloc[32]

# Generate Prompt using template
prompt = template.format(
    problem=row.problem,
    solution=""
)

# Infer
output = gemma_lm.generate(prompt, max_length=1024)

# Colorize
output = colorize_text(output)

# Display in markdown
display(Markdown(output))


# AIMO Data

So far we have inferred our model on **Math** dataset but now let's see how our model perform on AIMO (competition) dataset.


## Utilities


In [ ]:
import re

# Extract answer from model response
def get_answer(text):
    try:
        answer = re.search(r'Answer:\s*([\s\S]+)', text).group(1).strip()
        answer = answer.replace(",","")
        if is_integer(answer):
            return int(answer)%1000
        else:
            return 0
    except:
        return 0
    
    
def infer(df):
    preds = []
    for i in tqdm(range(len(df))):
        row = df.iloc[i]

        # Generate Prompt using template
        prompt = template.format(
            problem=row.problem,
            solution=""
        )

        # Infer
        output = gemma_lm.generate(prompt, max_length=1024)
        pred = get_answer(output)

        # Store predictions
        preds.append([row.id, pred])
        if "answer" in row:
            preds[-1] += [row.answer]
    return preds

## Inference on AIMO Data


In [ ]:
aimo_df = pd.read_csv(f"{CFG.dataset_path}/train.csv")
train_preds = infer(aimo_df)
train_pred_df = pd.DataFrame(train_preds, columns=["id", "prediction", "answer"])
train_pred_df

# Submission


## Infer on Test Data


In [ ]:
test_df = pd.read_csv(f"{CFG.dataset_path}/test.csv")
test_preds = infer(test_df)

## Prepare Submission File

While preparing the submission file, we must keep in mind that, the answer must be between `0-999`. This can easily handled by using `remainder (%)` operation. For this notebook, this step is already applied in the inference stage while extracting `answer` from `solution`. So, we don't need to separately apply it heer.


In [ ]:
sub_df = pd.DataFrame(test_preds, columns=["id", "answer"])
sub_df.to_csv("submission.csv",index=False,header=True)
sub_df.head()

# Conclusion

We can see that after fine-tuning, the model is following instructions more accurately. However, it may still struggle to solve problems accurately, which can be attributed to its small size. Nevertheless, there is ample room for improvement. Here are some tips to enhance performance:

- Train on the full data instead of first `1000` samples.
- Try using the non-instruction-tuned version of Gemma.
- Increase the `sequence_length`.
- Experiment with advanced prompt engineering techniques.
- Implement augmentation to increase the number of samples.
- Utilize a learning rate scheduler.


# Reference

- [Fine-tune Gemma models in Keras using LoRA](https://www.kaggle.com/code/nilaychauhan/fine-tune-gemma-models-in-keras-using-lora)
- [Parameter-efficient fine-tuning of GPT-2 with LoRA](https://keras.io/examples/nlp/parameter_efficient_finetuning_of_gpt2_with_lora/)
- [Gemma - KerasNLP](https://keras.io/api/keras_nlp/models/gemma/)
